In [10]:
!pip install requests

In [11]:
import pandas as pd
import numpy as np
import requests

## 1. Ouverture Fichiers

In [12]:
resultats = pd.read_csv('datas/resultats-par-niveau-cirlg-t1-france-entiere.csv')
display(resultats)

,Code du département,Libellé du département,Code de la circonscription,Libellé de la circonscription,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,Voix_11,% Voix/Ins_11,% Voix/Exp_11,N°Panneau_12,Sexe_12,Nom_12,Prénom_12,Voix_12,% Voix/Ins_12,% Voix/Exp_12
0,01,Ain,1,1ère circonscription,Complet,85723,18592,21.69,67131,78.31,...,437,0.51,0.67,12,M,DUPONT-AIGNAN,Nicolas,1753,2.04,2.67
1,01,Ain,2,2ème circonscription,Complet,99383,19917,20.04,79466,79.96,...,467,0.47,0.60,12,M,DUPONT-AIGNAN,Nicolas,1883,1.89,2.42
2,01,Ain,3,3ème circonscription,Complet,81500,20440,25.08,61060,74.92,...,424,0.52,0.71,12,M,DUPONT-AIGNAN,Nicolas,1820,2.23,3.04
3,01,Ain,4,4ème circonscription,Complet,94359,19826,21.01,74533,78.99,...,445,0.47,0.61,12,M,DUPONT-AIGNAN,Nicolas,2020,2.14,2.78
4,01,Ain,5,5ème circonscription,Complet,77144,18766,24.33,58378,75.67,...,399,0.52,0.70,12,M,DUPONT-AIGNAN,Nicolas,1522,1.97,2.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ,Français établis hors de France,7,7ème circonscription,Complet,122158,69559,56.94,52599,43.06,...,349,0.29,0.67,12,M,DUPONT-AIGNAN,Nicolas,639,0.52,1.23
573,ZZ,Français établis hors de France,8,8ème circonscription,Complet,130065,106455,81.85,23610,18.15,...,133,0.10,0.57,12,M,DUPONT-AIGNAN,Nicolas,300,0.23,1.28
574,ZZ,Français établis hors de France,9,9ème circonscription,Complet,121122,81344,67.16,39778,32.84,...,135,0.11,0.34,12,M,DUPONT-AIGNAN,Nicolas,381,0.31,0.97
575,ZZ,Français établis hors de France,10,10ème circonscription,Complet,103910,63575,61.18,40335,38.82,...,163,0.16,0.41,12,M,DUPONT-AIGNAN,Nicolas,530,0.51,1.33


In [13]:
positionnement = pd.read_csv('datas/positionnement_politique_v3.csv')
#display(positionnement)

## 2. Cleaning fichier 'positionnement'

##### Séparer noms et prénoms des candidats

In [14]:
positionnement[['prenom','nom']] = positionnement['nom_candidat'].str.split(n=1, expand=True)
positionnement['nom'] = positionnement['nom'].str.upper()
del positionnement['nom_candidat'] #to delete sepcific colomn

##### Réagencer les données

In [15]:
positionnement = positionnement.reindex(columns=['nom','prenom','parti_politique','sigle','positionnement','details_positionnement','tete_du_parti'])

## 3. Obtenir les numéros de panneaux associés aux candidat

In [16]:
candidat_by_colonne = [] #pour créer ma liste
panneau_num = []

for colonne in resultats.columns:
    if colonne.startswith('Nom_'):
        candidat = resultats[colonne].unique()
        candidat_by_colonne.extend(candidat)

for colonne in resultats.columns:
    if colonne.startswith('N°Panneau_'):
        num = resultats[colonne].unique()
        panneau_num.extend(num)

candidat_num = pd.DataFrame({'numéro':panneau_num,'nom':candidat_by_colonne})
#display(candidat_num)

## 4. Ajouter mes numéros à mon df 'Positionnement' 

In [25]:
num_et_bord = pd.merge(positionnement, candidat_num, on='nom', how='inner')
num_et_bord = num_et_bord[['numéro']+list(num_et_bord.columns[:-1])]
num_et_bord = num_et_bord.sort_values(by='positionnement') 
#display(num_et_bord)

In [49]:
#obtenir la liste des numéros de candidats de gauche, puis celles des candidats de droite

candidats_droite = num_et_bord[num_et_bord['positionnement'] == 'droite']['numéro'].astype(str).tolist()
candidats_gauche = num_et_bord[num_et_bord['positionnement'] == 'gauche']['numéro'].astype(str).tolist()

print('Droite : ', candidats_droite)
print('Gauche : ', candidats_gauche)

Droite :  ['12', '3', '10', '5', '6']
Gauche :  ['1', '8', '9', '7', '11', '2']


## 5. Résultats par circonscriptions

In [50]:
voix_candidats_droite = ['Voix_' + valeur for valeur in candidats_droite]
voix_candidats_gauche = ['Voix_' + valeur for valeur in candidats_gauche]

display(voix_exp_candidats_gauche)

['Voix_1', 'Voix_8', 'Voix_9', 'Voix_7', 'Voix_11', 'Voix_2']

In [54]:
# Fonction pour additionner les valeurs des colonnes spécifiées
def sommes_voix_gauche(circonscription):
    return circonscription[voix_candidats_gauche].sum()

# Appliquer la fonction à chaque ligne du DataFrame
resultats['Somme_Voix_Gauche'] = resultats.apply(sommes_voix_gauche, axis=1)

In [53]:
#ajouter aussi la somme voix de droites

def sommes_voix_droite(circonscription):
    return circonscription[voix_candidats_droite].sum()

resultats['Somme_Voix_Droite'] = resultats.apply(sommes_voix_droite, axis=1)

display(resultats)

,Code du département,Libellé du département,Code de la circonscription,Libellé de la circonscription,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,% Voix/Exp_11,N°Panneau_12,Sexe_12,Nom_12,Prénom_12,Voix_12,% Voix/Ins_12,% Voix/Exp_12,Somme_Voix_Gauche,Somme_Voix_Droite
0,01,Ain,1,1ère circonscription,Complet,85723,18592,21.69,67131,78.31,...,0.67,12,M,DUPONT-AIGNAN,Nicolas,1753,2.04,2.67,16998,46137
1,01,Ain,2,2ème circonscription,Complet,99383,19917,20.04,79466,79.96,...,0.60,12,M,DUPONT-AIGNAN,Nicolas,1883,1.89,2.42,20553,55190
2,01,Ain,3,3ème circonscription,Complet,81500,20440,25.08,61060,74.92,...,0.71,12,M,DUPONT-AIGNAN,Nicolas,1820,2.23,3.04,18278,39787
3,01,Ain,4,4ème circonscription,Complet,94359,19826,21.01,74533,78.99,...,0.61,12,M,DUPONT-AIGNAN,Nicolas,2020,2.14,2.78,16532,54023
4,01,Ain,5,5ème circonscription,Complet,77144,18766,24.33,58378,75.67,...,0.70,12,M,DUPONT-AIGNAN,Nicolas,1522,1.97,2.67,16726,37924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,ZZ,Français établis hors de France,7,7ème circonscription,Complet,122158,69559,56.94,52599,43.06,...,0.67,12,M,DUPONT-AIGNAN,Nicolas,639,0.52,1.23,18160,33405
573,ZZ,Français établis hors de France,8,8ème circonscription,Complet,130065,106455,81.85,23610,18.15,...,0.57,12,M,DUPONT-AIGNAN,Nicolas,300,0.23,1.28,5548,17578
574,ZZ,Français établis hors de France,9,9ème circonscription,Complet,121122,81344,67.16,39778,32.84,...,0.34,12,M,DUPONT-AIGNAN,Nicolas,381,0.31,0.97,17770,21039
575,ZZ,Français établis hors de France,10,10ème circonscription,Complet,103910,63575,61.18,40335,38.82,...,0.41,12,M,DUPONT-AIGNAN,Nicolas,530,0.51,1.33,11108,28140


In [57]:
def bord_candidat_favori(circonscription):
    colonnes_voix = voix_candidats_droite + voix_candidats_gauche
    candidat_voix_id = circonscription[colonnes_voix]
    colonne_voix_max = candidat_voix_id.idxmax()
    return colonne_voix_max

resultats['Bord_Candidat_Favori'] = resultats.apply(bord_candidat_favori, axis=1)

TypeError: reduction operation 'argmax' not allowed for this dtype